In [ ]:
pip install aif360['all']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 28.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install fairlearn

In [ ]:
# Core Python Libraries
import numpy as np
import pandas as pd

# Scikit-Learn: Preprocessing, Modeling, Evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# XGBoost
from xgboost import XGBClassifier

# AIF360: Datasets and Metrics
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric

# AIF360: Preprocessing Algorithms
from aif360.algorithms.preprocessing import (
    Reweighing,
    DisparateImpactRemover,
    LFR
)

# AIF360: In-processing Algorithms
from aif360.algorithms.inprocessing import (
    MetaFairClassifier,
    GerryFairClassifier,
    PrejudiceRemover,
    ExponentiatedGradientReduction,
    GridSearchReduction,
    ARTClassifier,
    AdversarialDebiasing
)

#AIF360: Post-processing Algorithms
from aif360.algorithms.postprocessing import (
    RejectOptionClassification,
    CalibratedEqOddsPostprocessing,
    EqOddsPostprocessing
)

# TensorFlow for AdversarialDebiasing
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()


In [ ]:
#data loading

In [ ]:
def load_and_preprocess_german_data():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"
    column_names = [
        "status", "duration", "credit_history", "purpose", "credit_amount",
        "savings", "employment", "installment_rate", "personal_status",
        "other_debtors", "residence_since", "property", "age",
        "other_installment_plans", "housing", "number_credits", "job",
        "people_liable", "telephone", "foreign_worker", "target"
    ]
    df = pd.read_csv(url, sep=' ', header=None, names=column_names)

    # Binary target: 1 for good, 0 for bad
    df['target'] = df['target'].map({1: 1, 2: 0})

    # Binary protected attribute: age >= 25 -> 1 (privileged), < 25 -> 0 (unprivileged)
    df['age_binary'] = df['age'].apply(lambda x: 1 if x >= 25 else 0)

    # One-hot encode categorical features
    df_encoded = pd.get_dummies(df.drop(columns=['target', 'age_binary', 'age']), drop_first=True)

    X = df_encoded
    y = df['target']
    protected = df['age_binary']

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test, prot_train, prot_test = train_test_split(
        X_scaled, y, protected, test_size=0.3, random_state=42, stratify=y
    )

    return X_train, X_test, y_train, y_test, prot_train, prot_test


In [ ]:

def load_german_credit_data():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"
    columns = [
        "status", "duration", "credit_history", "purpose", "credit_amount",
        "savings", "employment", "installment_rate", "personal_status",
        "other_debtors", "residence_since", "property", "age",
        "other_installment_plans", "housing", "number_credits", "job",
        "people_liable", "telephone", "foreign_worker", "target"
    ]
    df = pd.read_csv(url, sep=' ', header=None, names=columns)
    df['target'] = df['target'].map({1: 1, 2: 0})
    df['age_binary'] = df['age'].apply(lambda x: 1 if x >= 25 else 0)

    X_raw = df.drop(columns=['target', 'age', 'age_binary'])
    y = df['target'].values
    prot = df['age_binary'].values

    return X_raw, y, prot

def preprocess_data(X_raw, y, prot):
    cat_cols = X_raw.select_dtypes(include='object').columns.tolist()
    num_cols = X_raw.select_dtypes(exclude='object').columns.tolist()

    X_raw_train, X_raw_test, y_train, y_test, prot_train, prot_test = train_test_split(
        X_raw, y, prot, test_size=0.3, random_state=42, stratify=y
    )

    encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    X_train_cat = encoder.fit_transform(X_raw_train[cat_cols])
    X_test_cat = encoder.transform(X_raw_test[cat_cols])

    X_train_num = X_raw_train[num_cols].values
    X_test_num = X_raw_test[num_cols].values

    X_train = np.hstack((X_train_num, X_train_cat))
    X_test = np.hstack((X_test_num, X_test_cat))

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test, prot_train, prot_test

In [ ]:
def load_german_credit_data_inprocess():
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"
    columns = ["status", "duration", "credit_history", "purpose", "credit_amount",
               "savings", "employment", "installment_rate", "personal_status",
               "other_debtors", "residence_since", "property", "age",
               "other_installment_plans", "housing", "number_credits", "job",
               "people_liable", "telephone", "foreign_worker", "target"]

    df = pd.read_csv(url, sep=' ', header=None, names=columns)
    df['target'] = df['target'].map({1: 1, 2: 0})
    df['age_binary'] = df['age'].apply(lambda x: 1 if x >= 25 else 0)

    X_raw = df.drop(columns=['target', 'age', 'age_binary'])
    y = df['target'].values
    prot = df['age_binary'].values

    X_train_raw, X_test_raw, y_train, y_test, prot_train, prot_test = train_test_split(
        X_raw, y, prot, test_size=0.3, random_state=42, stratify=y
    )

    categorical_cols = X_raw.select_dtypes(include='object').columns.tolist()
    numeric_cols = X_raw.select_dtypes(exclude='object').columns.tolist()

    encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    X_train_cat = encoder.fit_transform(X_train_raw[categorical_cols])
    X_test_cat = encoder.transform(X_test_raw[categorical_cols])

    X_train_num = X_train_raw[numeric_cols].values
    X_test_num = X_test_raw[numeric_cols].values

    X_train = np.hstack((X_train_num, X_train_cat))
    X_test = np.hstack((X_test_num, X_test_cat))

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test, prot_train, prot_test


In [ ]:
#Utility Functions

In [ ]:
def train_baseline_model(X_train, y_train, sample_weight=None):
    model = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
    model.fit(X_train, y_train, sample_weight=sample_weight)
    return model


In [ ]:
def evaluate_fairness(y_true, y_pred, prot, X=None):
    df = pd.DataFrame(np.hstack((X, y_true[:, None], prot[:, None])),
                      columns=[f"x{i}" for i in range(X.shape[1])] + ['label', 'protected'])

    dataset_true = BinaryLabelDataset(df=df,
                                      label_names=["label"],
                                      protected_attribute_names=["protected"],
                                      favorable_label=1, unfavorable_label=0)

    pred_dataset = dataset_true.copy()
    pred_dataset.labels = y_pred.reshape(-1, 1)

    metric = ClassificationMetric(dataset_true, pred_dataset,
                                  privileged_groups=[{'protected': 1}],
                                  unprivileged_groups=[{'protected': 0}])

    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'disparate_impact': metric.disparate_impact(),
        'statistical_parity_difference': metric.statistical_parity_difference(),
        'equal_opportunity_difference': metric.equal_opportunity_difference()
    }


In [ ]:
def evaluate_model(model, X_test, y_test, prot_test):
    y_pred = model.predict(X_test)
    return evaluate_fairness(y_test, y_pred, prot_test, X_test)


In [ ]:
#XGboost
X_train, X_test, y_train, y_test, prot_train, prot_test = load_and_preprocess_german_data()
baseline_model = train_baseline_model(X_train, y_train)
baseline_metrics = evaluate_model(baseline_model, X_test, y_test.to_numpy(), prot_test.to_numpy())

baseline_metrics

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:46:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


{'accuracy': 0.7033333333333334,
 'disparate_impact': np.float64(0.8115043301614455),
 'statistical_parity_difference': np.float64(-0.14826339248170883),
 'equal_opportunity_difference': np.float64(-0.20484949832775923)}

In [ ]:
inprocess_results = {}

In [ ]:
##disparate impact

In [ ]:
def apply_disparate_impact_remover(X_train, y_train, prot_train, repair_level=1.0):
    df = pd.DataFrame(X_train)
    df['target'] = y_train.values
    df['protected'] = prot_train.values

    dataset = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=df,
        label_names=['target'],
        protected_attribute_names=['protected']
    )

    dir_remover = DisparateImpactRemover(repair_level=repair_level)
    repaired_dataset = dir_remover.fit_transform(dataset)

    X_repaired = pd.DataFrame(repaired_dataset.features)
    y_repaired = pd.Series(repaired_dataset.labels.ravel())
    prot_repaired = pd.Series(repaired_dataset.protected_attributes.ravel())

    return X_repaired, y_repaired, prot_repaired


In [ ]:
# Apply Disparate Impact Remover to both train and test sets
X_train_repaired, y_train_repaired, prot_train_repaired = apply_disparate_impact_remover(
    pd.DataFrame(X_train), y_train, prot_train
)

X_test_repaired, y_test_repaired, prot_test_repaired = apply_disparate_impact_remover(
    pd.DataFrame(X_test), y_test, prot_test
)

# Train on repaired training data
fair_model = train_baseline_model(X_train_repaired, y_train_repaired)

# Evaluate on repaired test data
fair_metrics = evaluate_model(
    fair_model,
    X_test_repaired.to_numpy(),
    y_test_repaired.to_numpy(),
    prot_test_repaired.to_numpy()
)
inprocess_results['Disparate_Impact_Remover']=fair_metrics
fair_metrics


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:46:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


{'accuracy': 0.7166666666666667,
 'disparate_impact': np.float64(0.9916013437849943),
 'statistical_parity_difference': np.float64(-0.006307291228660361),
 'equal_opportunity_difference': np.float64(-0.07316053511705678)}

In [ ]:
##LFR

In [ ]:
def apply_lfr(X_train, y_train, prot_train):
    df = pd.DataFrame(X_train)
    df['target'] = y_train.values
    df['protected'] = prot_train.values

    dataset = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=df,
        label_names=['target'],
        protected_attribute_names=['protected']
    )

    lfr = LFR(unprivileged_groups=[{'protected': 0}],
          privileged_groups=[{'protected': 1}],
          k=10, Ax=0.01, Ay=1.0, Az=0.1, verbose=0)


    lfr.fit(dataset)
    transformed_dataset = lfr.transform(dataset)

    X_transformed = pd.DataFrame(transformed_dataset.features)
    y_transformed = pd.Series(transformed_dataset.labels.ravel())
    prot_transformed = pd.Series(transformed_dataset.protected_attributes.ravel())

    return X_transformed, y_transformed, prot_transformed, lfr


In [ ]:
X_train_lfr, y_train_lfr, prot_train_lfr, lfr_model = apply_lfr(pd.DataFrame(X_train), y_train, prot_train)


In [ ]:
print("Unique labels in y_train_lfr:", np.unique(y_train_lfr))


Unique labels in y_train_lfr: [0. 1.]


In [ ]:
fair_lfr_model = train_baseline_model(X_train_lfr, y_train_lfr)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:46:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
# Apply LFR transformation on test set
df_test = pd.DataFrame(X_test)
df_test['target'] = y_test.values
df_test['protected'] = prot_test.values

test_dataset = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=df_test,
    label_names=['target'],
    protected_attribute_names=['protected']
)

transformed_test_dataset = lfr_model.transform(test_dataset)

X_test_lfr = pd.DataFrame(transformed_test_dataset.features)
y_test_lfr = pd.Series(transformed_test_dataset.labels.ravel())
prot_test_lfr = pd.Series(transformed_test_dataset.protected_attributes.ravel())

# Evaluate
fair_lfr_metrics = evaluate_model(
    fair_lfr_model,
    X_test_lfr.to_numpy(),
    y_test_lfr.to_numpy(),
    prot_test_lfr.to_numpy()
)
inprocess_results['LFR']=fair_lfr_metrics

fair_lfr_metrics


/usr/local/lib/python3.11/dist-packages/aif360/metrics/classification_metric.py:278: RuntimeWarning: invalid value encountered in scalar divide
  TPR=TP / P, TNR=TN / N, FPR=FP / N, FNR=FN / P,
/usr/local/lib/python3.11/dist-packages/aif360/metrics/classification_metric.py:279: RuntimeWarning: invalid value encountered in scalar divide
  GTPR=GTP / P, GTNR=GTN / N, GFPR=GFP / N, GFNR=GFN / P,


{'accuracy': 0.9966666666666667,
 'disparate_impact': np.float64(1.003968253968254),
 'statistical_parity_difference': np.float64(0.0039525691699604515),
 'equal_opportunity_difference': np.float64(0.0)}

In [ ]:
#reweighing

In [ ]:
def apply_reweighing(X_train, y_train, prot_train):
    df_train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1), prot_train.reshape(-1, 1))),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=df_train,
        label_names=["label"],
        protected_attribute_names=["protected"]
    )
    RW = Reweighing(unprivileged_groups=[{'protected': 0}], privileged_groups=[{'protected': 1}])
    bld_rw = RW.fit_transform(bld_train)

    return bld_rw.features, bld_rw.labels.ravel(), bld_rw.instance_weights


In [ ]:
X_train, X_test, y_train, y_test, prot_train, prot_test = preprocess_data(*load_german_credit_data())

X_rw, y_rw, sample_weights = apply_reweighing(X_train, y_train, prot_train)

expected_num_features = X_rw.shape[1]

if X_test.shape[1] < expected_num_features:
    padding = expected_num_features - X_test.shape[1]
    X_test_aligned = np.hstack((X_test, np.zeros((X_test.shape[0], padding))))
elif X_test.shape[1] > expected_num_features:
    X_test_aligned = X_test[:, :expected_num_features]
else:
    X_test_aligned = X_test

model_rw = train_baseline_model(X_rw, y_rw, sample_weight=sample_weights)

reweighing_metric = evaluate_model(model_rw, X_test_aligned, y_test, prot_test)

inprocess_results['reweighing'] = reweighing_metric
reweighing_metric


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [23:46:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


{'accuracy': 0.72,
 'disparate_impact': np.float64(0.8646235255208906),
 'statistical_parity_difference': np.float64(-0.10327138171726524),
 'equal_opportunity_difference': np.float64(-0.16095317725752512)}

In [ ]:
for model, metrics in inprocess_results.items():
    print(f"🔹 {model}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")
    print()


🔹 Disparate_Impact_Remover
  accuracy: 0.7167
  disparate_impact: 0.9916
  statistical_parity_difference: -0.0063
  equal_opportunity_difference: -0.0732

🔹 LFR
  accuracy: 0.9967
  disparate_impact: 1.0040
  statistical_parity_difference: 0.0040
  equal_opportunity_difference: 0.0000

🔹 reweighing
  accuracy: 0.7200
  disparate_impact: 0.8646
  statistical_parity_difference: -0.1033
  equal_opportunity_difference: -0.1610



In [ ]:
#Inprocess

In [ ]:
X_train, X_test, y_train, y_test, prot_train, prot_test = load_german_credit_data_inprocess()
results = {}

In [ ]:
def train_gerryfair(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train[:, None], prot_train[:, None])),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train, label_names=["label"], protected_attribute_names=["protected"],
                                   favorable_label=1, unfavorable_label=0)

    df_test = pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test, label_names=["label"], protected_attribute_names=["protected"],
                                  favorable_label=1, unfavorable_label=0)

    clf = GerryFairClassifier(C=100, printflag=False, gamma=0.005, fairness_def='FP', max_iters=50)
    clf.fit(bld_train)
    pred = clf.predict(bld_test)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [ ]:
results['GerryFair'] = train_gerryfair(X_train, y_train, prot_train, X_test, y_test, prot_test)


In [ ]:
def train_prejudice_remover(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1), prot_train.reshape(-1, 1))),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=1,
                                   unfavorable_label=0)

    df_test = pd.DataFrame(np.hstack((X_test, y_test.reshape(-1, 1), prot_test.reshape(-1, 1))),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=1,
                                  unfavorable_label=0)

    clf = PrejudiceRemover(sensitive_attr="protected", eta=25.0)
    clf.fit(bld_train)
    pred = clf.predict(bld_test)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [ ]:
results['PrejudiceRemover'] = train_prejudice_remover(X_train, y_train, prot_train, X_test, y_test, prot_test)


In [ ]:
def train_expgrad(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train[:, None], prot_train[:, None])),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=1,
                                   unfavorable_label=0)

    df_test = pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=1,
                                  unfavorable_label=0)

    expgrad = ExponentiatedGradientReduction(
        estimator=LogisticRegression(solver='liblinear'),
        constraints="DemographicParity"
    )
    expgrad.fit(bld_train)

    pred = expgrad.predict(bld_test)
    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [ ]:
results['ExponentiatedGradient'] = train_expgrad(X_train, y_train, prot_train, X_test, y_test, prot_test)


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
def train_gridsearch(X_train, y_train, prot_train, X_test, y_test, prot_test):
    df_train = pd.DataFrame(np.hstack((X_train, y_train[:, None], prot_train[:, None])),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=1,
                                   unfavorable_label=0)

    df_test = pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=1,
                                  unfavorable_label=0)

    grid = GridSearchReduction(
        estimator=LogisticRegression(solver='liblinear'),
        constraints="DemographicParity"
    )
    grid.fit(bld_train)

    pred = grid.predict(bld_test)
    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [ ]:
results['GridSearch'] = train_gridsearch(X_train, y_train, prot_train, X_test, y_test, prot_test)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
tf.disable_eager_execution()  # Required for TF1 compatibility in AIF360

def train_adversarial_debiasing(X_train, y_train, prot_train, X_test, y_test, prot_test):
    # Train data
    tf.reset_default_graph()
    df_train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1), prot_train.reshape(-1, 1))),
                            columns=[f"x{i}" for i in range(X_train.shape[1])] + ["label", "protected"])
    bld_train = BinaryLabelDataset(df=df_train,
                                   label_names=["label"],
                                   protected_attribute_names=["protected"],
                                   favorable_label=1,
                                   unfavorable_label=0)

    # Test data
    df_test = pd.DataFrame(np.hstack((X_test, y_test.reshape(-1, 1), prot_test.reshape(-1, 1))),
                           columns=[f"x{i}" for i in range(X_test.shape[1])] + ["label", "protected"])
    bld_test = BinaryLabelDataset(df=df_test,
                                  label_names=["label"],
                                  protected_attribute_names=["protected"],
                                  favorable_label=1,
                                  unfavorable_label=0)

    # TensorFlow session
    sess = tf.Session()

    clf = AdversarialDebiasing(
        privileged_groups=[{'protected': 1}],
        unprivileged_groups=[{'protected': 0}],
        scope_name='adv_debiasing',
        sess=sess,
        num_epochs=50,
        batch_size=64,
        debias=True
    )

    clf.fit(bld_train)
    pred = clf.predict(bld_test)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [ ]:
results['AdversarialDebiasing'] = train_adversarial_debiasing(X_train, y_train, prot_train, X_test, y_test, prot_test)


epoch 0; iter: 0; batch classifier loss: 0.804562; batch adversarial loss: 0.450733
epoch 1; iter: 0; batch classifier loss: 0.581346; batch adversarial loss: 0.450953
epoch 2; iter: 0; batch classifier loss: 0.415577; batch adversarial loss: 0.432432
epoch 3; iter: 0; batch classifier loss: 0.411867; batch adversarial loss: 0.476176
epoch 4; iter: 0; batch classifier loss: 0.418556; batch adversarial loss: 0.441483
epoch 5; iter: 0; batch classifier loss: 0.405587; batch adversarial loss: 0.479499
epoch 6; iter: 0; batch classifier loss: 0.444162; batch adversarial loss: 0.529955
epoch 7; iter: 0; batch classifier loss: 0.454269; batch adversarial loss: 0.545939
epoch 8; iter: 0; batch classifier loss: 0.396039; batch adversarial loss: 0.443662
epoch 9; iter: 0; batch classifier loss: 0.365126; batch adversarial loss: 0.493021
epoch 10; iter: 0; batch classifier loss: 0.356113; batch adversarial loss: 0.421430
epoch 11; iter: 0; batch classifier loss: 0.465698; batch adversarial loss:

In [ ]:
print("Fairness Evaluation Across Compatible AIF360 In-processing Algorithms\n")
for model, metrics in results.items():
    print(f"🔹 {model}")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")
    print()


Fairness Evaluation Across Compatible AIF360 In-processing Algorithms

🔹 GerryFair
  accuracy: 0.7500
  disparate_impact: 0.8344
  statistical_parity_difference: -0.1309
  equal_opportunity_difference: -0.1605

🔹 PrejudiceRemover
  accuracy: 0.7200
  disparate_impact: 1.0320
  statistical_parity_difference: 0.0244
  equal_opportunity_difference: -0.0017

🔹 ExponentiatedGradient
  accuracy: 0.7733
  disparate_impact: 1.0331
  statistical_parity_difference: 0.0259
  equal_opportunity_difference: 0.0263

🔹 GridSearch
  accuracy: 0.7633
  disparate_impact: 1.0177
  statistical_parity_difference: 0.0140
  equal_opportunity_difference: 0.0263

🔹 AdversarialDebiasing
  accuracy: 0.7233
  disparate_impact: 1.1057
  statistical_parity_difference: 0.0773
  equal_opportunity_difference: 0.0146



In [ ]:
#postprocessing

In [ ]:
X_train, X_test, y_train, y_test, prot_train, prot_test = load_german_credit_data_inprocess()
post_results = {}

In [ ]:
def train_roc_postprocessing_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test):
    model = train_baseline_model(X_train, y_train)

    y_prob = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)

    bld_test = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                        columns=[f'x{i}' for i in range(X_test.shape[1])] + ['label', 'protected']),
        label_names=['label'],
        protected_attribute_names=['protected']
    )

    bld_pred = bld_test.copy()
    bld_pred.scores = y_prob.reshape(-1, 1)
    bld_pred.labels = y_pred.reshape(-1, 1)

    roc = RejectOptionClassification(
        unprivileged_groups=[{'protected': 0}],
        privileged_groups=[{'protected': 1}],
        low_class_thresh=0.01, high_class_thresh=0.99,
        num_class_thresh=100, num_ROC_margin=50,
        metric_name="Statistical parity difference",
        metric_ub=0.05, metric_lb=-0.05
    )
    roc = roc.fit(bld_test, bld_pred)
    pred = roc.predict(bld_pred)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [ ]:
post_results['RejectOptionClassification'] = train_roc_postprocessing_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test)


In [ ]:
def train_calibrated_eq_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test):
    model = train_baseline_model(X_train, y_train)

    y_prob = model.predict_proba(X_test)[:, 1]

    bld_test = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                        columns=[f'x{i}' for i in range(X_test.shape[1])] + ['label', 'protected']),
        label_names=['label'],
        protected_attribute_names=['protected']
    )

    bld_pred = bld_test.copy()
    bld_pred.scores = y_prob.reshape(-1, 1)

    ceo = CalibratedEqOddsPostprocessing(
        privileged_groups=[{'protected': 1}],
        unprivileged_groups=[{'protected': 0}],
        cost_constraint="fnr",
        seed=42
    )
    ceo = ceo.fit(bld_test, bld_pred)
    pred = ceo.predict(bld_pred)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [ ]:
post_results['CalibratedEqOdds'] = train_calibrated_eq_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test)


In [ ]:
def train_equalized_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test):
    model = train_baseline_model(X_train, y_train)

    y_pred = model.predict(X_test)

    bld_test = BinaryLabelDataset(
        favorable_label=1,
        unfavorable_label=0,
        df=pd.DataFrame(np.hstack((X_test, y_test[:, None], prot_test[:, None])),
                        columns=[f'x{i}' for i in range(X_test.shape[1])] + ['label', 'protected']),
        label_names=['label'],
        protected_attribute_names=['protected']
    )

    bld_pred = bld_test.copy()
    bld_pred.labels = y_pred.reshape(-1, 1)

    eq = EqOddsPostprocessing(
        privileged_groups=[{'protected': 1}],
        unprivileged_groups=[{'protected': 0}]
    )
    eq = eq.fit(bld_test, bld_pred)
    pred = eq.predict(bld_pred)

    return evaluate_fairness(y_test, pred.labels.ravel(), prot_test, X_test)


In [ ]:
post_results['EqualizedOdds'] = train_equalized_odds_with_xgb(X_train, y_train, prot_train, X_test, y_test, prot_test)

In [ ]:
for model, metrics in post_results.items():
    print(f"🔹 {model}")
    for k, v in metrics.items():
        print(f"  {k}: {v:.4f}")
    print()


🔹 RejectOptionClassification
  accuracy: 0.7533
  disparate_impact: 1.0525
  statistical_parity_difference: 0.0372
  equal_opportunity_difference: 0.0146

🔹 CalibratedEqOdds
  accuracy: 0.7067
  disparate_impact: 0.6596
  statistical_parity_difference: -0.3404
  equal_opportunity_difference: -0.2692

🔹 EqualizedOdds
  accuracy: 0.6167
  disparate_impact: 0.9590
  statistical_parity_difference: -0.0282
  equal_opportunity_difference: 0.0188

